Importing the libraries 

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

Importing the dataset

Unzipping the data first 

In [2]:
!unzip '/content/drive/MyDrive/Deep Learning/train.csv.zip'

Archive:  /content/drive/MyDrive/Deep Learning/train.csv.zip
  inflating: train.csv               


In [3]:
data = pd.read_csv('/content/train.csv')

In [4]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Drop the rows with NaN values

In [5]:
data = data.dropna()

Getting the independent and dependent variables

In [6]:
X = data.drop('label', axis=1)
y = data['label']

Checking the X data and y data shape

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

Importing tensorflow 

In [9]:
import tensorflow as tf 

Checking the tensorflow version

In [10]:
print(tf.__version__)

2.8.2


Importing LSTM libraries

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

Defining the max vocabulary size

In [12]:
vocabsize = 5000

One hot representation 

In [13]:
messages = X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

Resetting indexes

In [15]:
messages.reset_index(inplace=True)

Importing the nltk libraries

In [16]:
import nltk 
import re 
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Data Preprocessing

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Checking the corpus

In [19]:
corpus[:10]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag']

One Hot Represenation for the corpus 

In [20]:
onehot = [one_hot(words, vocabsize) for words in corpus]
onehot[:5]

[[1547, 1040, 3573, 2432, 4171, 3534, 881, 642, 1772, 2077],
 [76, 2791, 3245, 3181, 3457, 1976, 4410],
 [1146, 4736, 1531, 3144],
 [2112, 2667, 3415, 16, 2497, 2436],
 [3346, 3457, 2763, 3610, 1280, 1558, 3457, 3747, 2832, 1921]]

Embedding Representation

In [21]:
sentencelength = 20 
embeddeddocs = pad_sequences(onehot, padding='pre', maxlen=sentencelength)
print(embeddeddocs)

[[   0    0    0 ...  642 1772 2077]
 [   0    0    0 ... 3457 1976 4410]
 [   0    0    0 ... 4736 1531 3144]
 ...
 [   0    0    0 ...   96 2224 2471]
 [   0    0    0 ... 2967 1233 1372]
 [   0    0    0 ... 1011  686 1488]]


In [22]:
embeddeddocs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1547,
       1040, 3573, 2432, 4171, 3534,  881,  642, 1772, 2077], dtype=int32)

Creating the model

In [32]:
from nltk import metrics
embedded_vecor_features = 40
modell = Sequential()
modell.add(Embedding(vocabsize, embedded_vecor_features, input_length=sentencelength))
modell.add(LSTM(100))
modell.add(Dense(1, activation='sigmoid'))
modell.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
print(modell.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
len(embeddeddocs), y.shape

(18285, (18285,))

Import libraries

In [34]:
import numpy as np
Xfinal = np.array(embeddeddocs)
yfinal = np.array(y)

Checking the shape of Xfinal and yfinal

In [35]:
print(Xfinal.shape)
print(yfinal.shape)

(18285, 20)
(18285,)


Train test split

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xfinal, yfinal, test_size=0.33, random_state=42)

Training the model

In [37]:
modell.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 42ms/step - loss: 0.3231 - accuracy: 0.8537 - val_loss: 0.1947 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 8s 40ms/step - loss: 0.1370 - accuracy: 0.9474 - val_loss: 0.1985 - val_accuracy: 0.9190
Epoch 3/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0958 - accuracy: 0.9651 - val_loss: 0.2411 - val_accuracy: 0.9186
Epoch 4/10
192/192 [==============================] - 9s 46ms/step - loss: 0.0674 - accuracy: 0.9764 - val_loss: 0.2365 - val_accuracy: 0.9135
Epoch 5/10
192/192 [==============================] - 8s 41ms/step - loss: 0.0449 - accuracy: 0.9851 - val_loss: 0.3392 - val_accuracy: 0.9062
Epoch 6/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0296 - accuracy: 0.9905 - val_loss: 0.3405 - val_accuracy: 0.9104
Epoch 7/10
192/192 [==============================] - 7s 38ms/step - loss: 0.0168 - accuracy: 0.9953 - val_loss: 0.4703 - val_accuracy: 0.904

Performance Metrics Accuracy

In [44]:
y_pred = (modell.predict(X_test) > 0.5).astype("int32")

Confusion Matrix

In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3125,  294],
       [ 272, 2344]])

Accuracy Score

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9062137531068766

Classification Report 

In [47]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3419
           1       0.89      0.90      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.90      6035
weighted avg       0.91      0.91      0.91      6035

